# Cleaning file Script

### Purpose of the script

The purpose of this script is to clean all the files composing our database. This will be the basis of our project to create all the dimensions, and fact tables.

In [1]:
import pandas as pd
import numpy as np
import random as rd
pd.options.display.max_columns = 400
print('The pandas version is {}.'.format(pd.__version__))
import hashlib
from rdgeneration import Rdgen
pd.options.mode.chained_assignment = None 

The pandas version is 0.23.4.


ModuleNotFoundError: No module named 'rdgeneration'

### File format

In [ ]:
df = pd.read_excel('./table/2015/BDN_interpA-A1-S2-2015.xlsx')

In [ ]:
df

<div class="alert alert-info">
Every excel files that will be added to the database has to follow the format bellow:
    <ul>
        <li>line: 1-3 $\rightarrow$ empty lines </li>
        <li>line 4:
            <ul>
                <li>column 4 : Programme(s) / Période (s) </li>
                <li>column 6 : answer </li>
            </ul>
        </li>
        <li>line 6:
            <ul>
                <li>column 4 : Code UE </li>
                <li>column 6 : answer </li>
            </ul>
        </li>
        <li>line 7:
            <ul>
                <li>column 4 : Libellé UE </li>
                <li>column 6 : answer </li>
            </ul>
        </li>
        <li>line 8:
            <ul>
                <li>column 4 : Responsable UE</li>
                <li>column 6 : answer </li>
            </ul>
        </li>
        <li>line 10:
            <ul>
                <li>column 4 : Remise limite CC</li>
                <li>column 6 : answer </li>
            </ul>
        </li>
        <li>line 11:
            <ul>
                <li>column 4 : Remise limite Exam </li>
                <li>column 6 : answer </li>
                <li>column 9-18: Code UV</li>
            </ul>
        </li>
        <li>line 12:
            <ul>
                <li>column 9-18: Code UV</li>
            </ul>
        </li>
        <li>line 13:
            <ul>
                <li>column 1 : N° </li>
                <li>column 4 : N° Etudiant </li>
                <li>column 5 : Nom</li>
                <li>column 6 : Prénom</li>
                <li>column 7 : Groupe</li>
                <li>column 8 : Intervenant</li>
                <li>column 9-18 : Pondération</li>
                <li>column 19 : Moyenne</li>
                <li>column 20 : Grade</li>
                <li>column 21 : Grade ECTS</li>
                <li>column 22 : Moyenne saisie</li>
                <li>column 23 : Grade saisie</li>
                <li>column 24 : Commentaires</li>
            </ul>
        </li>
        <li>
            line 14-the end: data
        </li>
    </ul>
</div>

In [ ]:
df.head(5)

The column for each competence are:
- numero
- numero_Etudiant
- nom
- prenom


- nom_cours
- note_cours
- moyenne
- grade_atteint
- grade_ects


- code_UE
- libelle_UE
- responsable_UE
- annee

In [ ]:
mycol = ["numero","numero_etudiant","nom","prenom",
        "nom_cours","note_cours","moyenne","grade_atteint",
        "grade_ects","code_UE","libelle_UE","responsable_UE"]

#### Column numero
- numero

In [ ]:
num_col = pd.to_numeric(df["Unnamed: 0"].loc[11:])
max_num = num_col.max()
ind_max = num_col[num_col == max_num].index[0]

In [ ]:
num_col = num_col.loc[:ind_max].apply(int)
number_std = len(num_col)

The numero column : **num_col**

### Filling missing columns with randomly generated values
- numero_Etudiant
- nom
- prenom

In [ ]:
rd_val = Rdgen(number_std)

In [ ]:
firstname_col = rd_val.get_firstname()
lastname_col = rd_val.get_lastname()
std_num_col = rd_val.get_std_num()

The firstname column: **firstname_col** <br>
The lastname column: **lastname_coll** <br>
the student number column: **std_num_col** <br>

### General values
- code_UE
- libelle_UE
- responsable_UE
- annee

In [ ]:
code_UE = df["Relevé de notes"].iloc[3]
libelle_UE = df["Relevé de notes"].iloc[4]
responsable_UE = df["Relevé de notes"].iloc[5]
annee = df["Relevé de notes"].iloc[1]

In [ ]:
print("code_UE : {}\nlibelle_UE : {}\nresponsable_UE : {}\nannee :{}".format(code_UE,libelle_UE,responsable_UE,annee))

### Dealing with UV columns
- nom_cours
- note_cours
- moyenne
- grade_atteint
- grade_ects

### Preparing the dataset

We can drop the first 10 columns since we already have the information

In [ ]:
col_todrop = list(df.loc[:,"Unnamed: 24":].columns)

In [ ]:
Uv_name = list(df.loc[9,"Unnamed: 8":"Unnamed: 17"])

In [ ]:
for k in range(len(Uv_name)):
    if(Uv_name[k] == "Niveau-Situation"):
        Uv_name[k] = "Niveau-Situation{}".format(k)

In [ ]:
df.drop(col_todrop,axis=1,inplace=True)

In [ ]:
df = df.iloc[10:]
df_col = list(df.iloc[0])


In [ ]:
df_col = df_col[0:8]+Uv_name+df_col[18:]

In [ ]:
df.columns = df_col
df.drop(10,inplace = True)
df.reset_index(inplace=True)
df.drop(['IDOPUSER','IDOPSESSION',"index","Groupe","Intervenant"], axis =1, inplace= True)

In [ ]:
df = df.iloc[:number_std]

### Adding the generated information

In [ ]:
df

In [ ]:
df["N° Etudiant"]= std_num_col
df["Nom"] = lastname_col
df["Prénom"]= firstname_col

In [ ]:
UV = df.loc[:,"Prénom":"Moyenne"].drop(["Prénom","Moyenne"],axis = 1)

In [ ]:

def drop_na_line(var):
    if(list(var) == [True, True, True, True, True, True, True, True, True, True]):
        df.drop(var.name,inplace=True)
    return

def drop_na_col(var):
    tocompare = [True]*len(df)
    if(list(var) == tocompare):
        df.drop(var.name,inplace = True, axis = 1)

In [ ]:
c = UV.isna().apply(lambda var : drop_na_line(var),axis=1)

In [ ]:
c = df.isna().apply(lambda var : drop_na_col(var),axis=0)

In [ ]:
code_UE = df["Relevé de notes"].iloc[3]
libelle_UE = df["Relevé de notes"].iloc[4]
responsable_UE = df["Relevé de notes"].iloc[5]
annee = df["Relevé de notes"].iloc[1]

In [ ]:
df["code_UE"] = code_UE
df["libelle_UE"] = libelle_UE
df["responsable_UE"] = responsable_UE
df["annee"] = annee

In [ ]:
Uv_name = list(df.loc[:,"Prénom":"Moyenne"])
Uv_name.pop(0)
Uv_name.pop(-1)

In [ ]:
dict_uv = {}
for k in Uv_name:
    dict_uv[k] = df[k]
dict_uv

In [ ]:
df.head()